In [2]:
import pandas as pd
housing_data = pd.read_csv('train.csv')

housing_data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = housing_data.SalePrice
X = housing_data.drop(columns = 'SalePrice')

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state = 0)

cat_cols = [c for c in X_train.columns if
             X_train[c].dtype == "object" ]

num_cols = [i for i in X_train.columns if
             X_train[i].dtype in ['int64', 'float64']]

my_cols = cat_cols + num_cols

X.head()

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

num_trans = SimpleImputer(strategy='constant')

cat_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
     ])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_trans, num_cols),
        ('cat', cat_trans, cat_cols)
    ])

model = XGBRegressor(n_estimators = 500, learning_rate = 0.05)

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

clf.fit(X_train, y_train)

preds = clf.predict(X_valid)


print('MAE:', mean_absolute_error(y_valid, preds))

housing_tfile = 'test.csv'
housing_tdata = pd.read_csv(housing_tfile)
tX = housing_tdata

pred = clf.predict(tX)

output = pd.DataFrame({'Id': tX.Id,
                       'SalePrice': pred})
output.to_csv('submission.csv', index=False)


MAE: 15997.645333904109
